<a href="https://colab.research.google.com/github/DGuilherme/TurbofanVibration/blob/master/KerasRegressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keras Regressor 


*usefull links*  
* [Adam](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/)
+ [Keras Regressor](https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/)

In [60]:
# Use seaborn for pairplot
!pip install -q seaborn

In [61]:
# imports
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# random forests
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,accuracy_score,r2_score,mean_absolute_error



In [62]:
# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

In [63]:
# Load the data

###################
#      TRAIN      #
###################

url = 'https://raw.githubusercontent.com/DGuilherme/TurbofanVibration/master/Dataset/ALLtrainMescla5D.csv'
train_data = pd.read_csv(url,delimiter=r",")
#print(train_data)

model_mapping  ={'model1': 1, 'model2': 2, 'model3': 3, 'model4': 4}
train_data = train_data.replace({'model': model_mapping})

failure_mapping = {'none': 0, 'comp1': 1, 'comp2': 2, 'comp3': 3, 'comp4': 4}
train_data = train_data.replace({'failure': failure_mapping})

# remover colunas
train_data = train_data.drop('datetime',axis=1)
train_data = train_data.astype('float32')


n_features = 31
n_target = 35
target = 'RUL_I'

feature_list = [train_data.columns[i] for i in range(0,n_features)]

X_train = train_data.iloc[:, 0:n_features].values.astype('float32')

# generate labels
Y_train = train_data.iloc[:, n_target-1:n_target].values.astype('float32').ravel()

###################
#      TEST       #
###################

#https://raw.githubusercontent.com/DGuilherme/TurbofanVibration/master/Dataset/ALLtrainMescla5D.csv

url = 'https://raw.githubusercontent.com/DGuilherme/TurbofanVibration/master/Dataset/ALLtrainMescla5D.csv'
test_data = pd.read_csv(url,delimiter=r",")

model_mapping  ={'model1': 1, 'model2': 2, 'model3': 3, 'model4': 4}
test_data = test_data.replace({'model': model_mapping})

failure_mapping = {'none': 0, 'comp1': 1, 'comp2': 2, 'comp3': 3, 'comp4': 4}
test_data = test_data.replace({'failure': failure_mapping})

# remover colunas
test_data = test_data.drop('datetime',axis=1)
test_data = test_data.astype('float32')


X_test = test_data.iloc[:, 0:n_features].values.astype('float32')
# generate labels
Y_test = test_data.iloc[:, n_target-1:n_target].values.astype('float32').ravel()



In [64]:
from sklearn.preprocessing import MinMaxScaler

num_cols = train_data.columns[train_data.dtypes.apply(lambda c: np.issubdtype(c, np.number))]

trans = MinMaxScaler()
# normaliza train
train_data[num_cols] = trans.fit_transform(train_data[num_cols])

#normaliza test
test_data[num_cols] = trans.fit_transform(test_data[num_cols])

## Remove features
train_useless_features = train_data[['DI','RUL']]
train_data = train_data.drop(['DI','RUL'],axis=1)

train_RUL_I = train_data["RUL_I"]
train_data = train_data.drop("RUL_I",axis=1)

test_useless_features = test_data[['DI','RUL']]
test_data = test_data.drop(['DI','RUL'],axis=1)

test_RUL_I = test_data["RUL_I"]
test_data = test_data.drop("RUL_I",axis=1)

In [65]:
from sklearn.model_selection import train_test_split
# split the dataset in evaluation and test set
X_eva, X_test, Y_eva, Y_test = train_test_split(test_data, test_RUL_I, test_size=0.77, random_state=42)
print("Number of samples for evaluation: %d, Number of features used: %d " % (X_eva.shape[0], X_eva.shape[1])) 
print("Number of samples for testing: %d, Number of features used: %d " % (X_test.shape[0], X_test.shape[1])) 


Number of samples for evaluation: 4799, Number of features used: 32 
Number of samples for testing: 16068, Number of features used: 32 


In [66]:
from keras.layers import Dense
# create a sequential model using tensor flow;
# Another way, is a functional model.
# add layers to the sequential model
def baseline_model():
  Telemetry_Vibration_NN = tf.keras.models.Sequential()


  # Input Layer: feed it with a tensor of size (30,1)
  Telemetry_Vibration_NN.add(Dense(32, input_dim = 32, kernel_initializer='normal', activation='relu' ))

  # Hidden Layer : It is composed by 15 neurons
  Telemetry_Vibration_NN.add(Dense(16, kernel_initializer='normal'))

  # Output Layer: 2 neurons, each one computes a probability of a particular classe
  # Telemetry_Vibration_NN.add(layers.Dense(1, activation = "softmax"))
  Telemetry_Vibration_NN.add(Dense(1, kernel_initializer='normal'))


  # display a summary of the model's topology and the parameters that need to be trained.
  Telemetry_Vibration_NN.summary()

  #Configure the model for training stage.
  Telemetry_Vibration_NN.compile(optimizer="adam", loss="mean_squared_error")
return Telemetry_Vibration_NN



Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_27 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 17        
Total params: 1,601
Trainable params: 1,601
Non-trainable params: 0
_________________________________________________________________


In [67]:
# Estimators
from sklearn.preprocessing import StandardScaler
from keras.wrappers.scikit_learn import KerasRegressor

#estimators = []
#estimators.append(('standardize', StandardScaler()))

# epochs - numero de interações
#estimators.append(('mlp', KerasRegressor(build_fn=Telemetry_Vibration_NN, epochs=100, batch_size=5, verbose=0)))
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)

In [69]:
# Pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

#pipeline = Pipeline(estimators)
kfold = KFold(n_splits=2)

# resultados 
results = cross_val_score(estimator, train_data, train_RUL_I, cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Baseline: nan (nan) MSE


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: The first argument to `Layer.call` must always be passed.

  FitFailedWarning)
